# **Finding Lane Lines on the Road** 
***
In this project, you will use the tools you learned about in the lesson to identify lane lines on the road.  You can develop your pipeline on a series of individual images, and later apply the result to a video stream (really just a series of images). Check out the video clip "raw-lines-example.mp4" (also contained in this repository) to see what the output should look like after using the helper functions below. 

Once you have a result that looks roughly like "raw-lines-example.mp4", you'll need to get creative and try to average and/or extrapolate the line segments you've detected to map out the full extent of the lane lines.  You can see an example of the result you're going for in the video "P1_example.mp4".  Ultimately, you would like to draw just one line for the left side of the lane, and one for the right.

---
Let's have a look at our first image called 'test_images/solidWhiteRight.jpg'.  Run the 2 cells below (hit Shift-Enter or the "play" button above) to display the image.

**Note** If, at any point, you encounter frozen display windows or other confounding issues, you can always start again with a clean slate by going to the "Kernel" menu above and selecting "Restart & Clear Output".

---

**The tools you have are color selection, region of interest selection, grayscaling, Gaussian smoothing, Canny Edge Detection and Hough Tranform line detection.  You  are also free to explore and try other techniques that were not presented in the lesson.  Your goal is piece together a pipeline to detect the line segments in the image, then average/extrapolate them and draw them onto the image for display (as below).  Once you have a working pipeline, try it out on the video stream below.**

---

<figure>
 <img src="line-segments-example.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your output should look something like this (above) after detecting line segments using the helper functions below </p> 
 </figcaption>
</figure>
 <p></p> 
<figure>
 <img src="laneLines_thirdPass.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your goal is to connect/average/extrapolate line segments to get output like this</p> 
 </figcaption>
</figure>

In [ ]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
%matplotlib inline

In [ ]:
#reading in an image
image = mpimg.imread('test_images/solidWhiteRight.jpg')
#printing out some stats and plotting
print('This image is:', type(image), 'with dimesions:', image.shape)
plt.imshow(image)  #call as plt.imshow(gray, cmap='gray') to show a grayscaled image

**Some OpenCV functions (beyond those introduced in the lesson) that might be useful for this project are:**

`cv2.inRange()` for color selection  
`cv2.fillPoly()` for regions selection  
`cv2.line()` to draw lines on an image given endpoints  
`cv2.addWeighted()` to coadd / overlay two images
`cv2.cvtColor()` to grayscale or change color
`cv2.imwrite()` to output images to file  
`cv2.bitwise_and()` to apply a mask to an image

**Check out the OpenCV documentation to learn about these and discover even more awesome functionality!**

Below are some helper functions to help get you started. They should look familiar from the lesson!

In [ ]:
import math

def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    you should call plt.imshow(gray, cmap='gray')"""
    #return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Or use BGR2GRAY if you read an image with cv2.imread()
    return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image
 
def draw_lines(img, lines, color=[255, 0, 0], thickness=2):
    """
    NOTE: this is the function you might want to use as a starting point once you want to 
    average/extrapolate the line segments you detect to map out the full
    extent of the lane (going from the result shown in raw-lines-example.mp4
    to that shown in P1_example.mp4).  
    
    Think about things like separating line segments by their 
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then, you can average the position of each of 
    the lines and extrapolate to the top and bottom of the lane.
    
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """
    for line in lines:
        for x1,y1,x2,y2 in line:
            cv2.line(img, (x1, y1), (x2, y2), color, thickness)

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img

# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)

## Test on Images

Now you should build your pipeline to work on the images in the directory "test_images"  
**You should make sure your pipeline works well on these images before you try the videos.**

In [ ]:
import os
os.listdir("test_images/")

run your solution on all test_images and make copies into the test_images directory).

In [ ]:
# TODO: Build your pipeline that will draw lane lines on the test_images
# then save them to the test_images directory.

DEBUG = False #if we are in debug, we will draw additional element on the image
RED   = [255,   0,   0]
GREEN = [0,   255,   0]
BLUE  = [0,     0, 255]
WHITE = [255, 255, 255]

def hsv(image):
    """convert image to hsv"""
    
    # Convert BGR to HSV
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    #define masks
    white_mask = cv2.inRange(hsv, np.array([0,0,225]), np.array([255,30,255]))    
    yellow_mask = cv2.inRange(hsv, np.array([80,100,100]), np.array([100,255,255]))
    
    # Bitwise-AND mask and original image
    white_result = cv2.bitwise_and(image, image, mask = white_mask)    
    yellow_result = cv2.bitwise_and(image, image, mask = yellow_mask)
    
    #combine the two result with or operation
    return cv2.bitwise_or(white_result, yellow_result)
    

def classify_lines(image, lines):
    """classify lines into left lines, right lines, and outliers lines"""
    mid_width = image.shape[1] / 2
    left = []
    right = []
    outliers = []
    for line in lines:
        for x1,y1,x2,y2 in line:
            #skip vertial line with infinite slope
            if x2 == x1:
                outliers.append((x1,y1,x2,y2,"NaN"))
                continue
            
            #calcuate slope
            slope = (y2-y1)/(x2-x1)
            
            #exclude lines with extreme slopes
            if slope < 0.3 and slope > -0.3:
                outliers.append((x1,y1,x2,y2,slope))
                continue

            #classify whether the line should be left or right using avg x
            avg_x = (x1 + x2) / 2
            
            #exclude lines with bad opposite slope on the left side (\)
            if avg_x <= mid_width and slope > 0:
                outliers.append((x1,y1,x2,y2,slope))
                continue
                
            #exclude lines with bad opposite slope on the left side (\) 
            if avg_x >= mid_width and slope < 0:
                outliers.append((x1,y1,x2,y2,slope))
                continue
            
            if avg_x <= mid_width:
                left.append((x1,y1,x2,y2,slope))
            elif avg_x >= mid_width:
                right.append((x1,y1,x2,y2,slope))
                
    return (left,right,outliers)
    
def fit_line(image, lines):
    """Create a fitted line"""
    if len(lines) == 0:
        return ()
       
    height = image.shape[0]
    #calculate the average for the coordinates and slope
    slope = sum(line[4] for line in lines) / len(lines)   
    x = sum(line[0] for line in lines) / len(lines)
    y = sum(line[1] for line in lines) / len(lines)
    
    by = height
    bx = x - (y - by) / slope
    ty = height * 3/5
    tx = x + (ty - y) / slope 
    return (int(bx), int(by), int(tx), int(ty), slope)

def get_region(img):
    """Returns the region of interest"""
    w = img.shape[1]
    h = img.shape[0]
    tl = ( int(w * 5 / 11),  int(h * 3 / 5) )
    tr = ( int(w * 6 / 11),  int(h * 3 / 5) )
    bl = ( 0, h)
    br = ( w, h)
    return [bl,tl,tr,br]

def get_hough_lines(img):
    """Returns lines detected by hough transform"""
    rho = 1 # distance resolution in pixels of the Hough grid
    theta = np.pi/180 # angular resolution in radians of the Hough grid
    threshold = 10   # minimum number of votes (intersections in Hough grid cell)
    min_line_len = 25 #minimum number of pixels making up a line
    max_line_gap = 15 # maximum gap in pixels between connectable line segments
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)        
    return lines

def show_image(image, title):
    """show image"""
    plt.figure()
    plt.title(title)
    if len(image.shape) <= 2:
        #use grayscale if no color channels
        plt.imshow(image, cmap='gray')
    else:
        plt.imshow(image)

def combine(image1, image2, image3, image4):
    #make sure all images have the same dimension, it's necessary to combine them
    #if image is grayscale (1 color channel), convert to rgb (3 color) 
    r1 = cv2.cvtColor(image1,cv2.COLOR_GRAY2RGB) if len(image1.shape) <= 2 else image1
    r2 = cv2.cvtColor(image2,cv2.COLOR_GRAY2RGB) if len(image2.shape) <= 2 else image2
    r3 = cv2.cvtColor(image3.cvt.COLOR_GRAY2RGB) if len(image3.shape) <= 2 else image3
    r4 = cv2.cvtColor(image4.cvt.COLOR_GRAY2RGB) if len(image4.shape) <= 2 else image4
        
    #shrink each image by a half on x and y direction
    r1 = cv2.resize(r1, None, fx=0.5, fy=0.5, interpolation = cv2.INTER_AREA)
    r2 = cv2.resize(r2, None, fx=0.5, fy=0.5, interpolation = cv2.INTER_AREA)
    r3 = cv2.resize(r3, None, fx=0.5, fy=0.5, interpolation = cv2.INTER_AREA)
    r4 = cv2.resize(r4, None, fx=0.5, fy=0.5, interpolation = cv2.INTER_AREA)
    
    #combine images 1,2 and 3,4 horizontally
    hcombined1 = np.concatenate((r1, r2), axis=1)
    hcombined2 = np.concatenate((r3, r4), axis=1)   
    #combine the result vertically
    vcombined = np.concatenate((hcombined1, hcombined2), axis=0)
    
    return vcombined
        
def process(image):
    #perform gray scale transform
    #gray = grayscale(image)
    gray = hsv(image)
    
    #perform gaussian blur to smooth the image
    blur_gray = gaussian_blur(gray, 7)
    
    #peform canny edge detection
    edges = canny(blur_gray, 50, 150)
    
    #limit to region of interest, get_region returns four corners of a quadrilateral 
    bl,tl,tr,br = get_region(image)
    masked_edges = region_of_interest(edges, np.array([[bl,tl,tr,br]], dtype=np.int32))
        
    #find line segments using hough transform
    lines = get_hough_lines(masked_edges)
    
    if lines == None or len(lines) == 0:
        #cannot find any lines
        print("cannot find any lines after hough transform.")
        return image

    #classify the lines into right, left and outliers
    left_lines, right_lines, outliers = classify_lines(image, lines)
    
    #fit the lines and draw it
    line_image = np.zeros((image.shape[0], image.shape[1], 3), dtype=np.uint8)
            
    if len(left_lines) > 0:
        left_fitted = fit_line(image, left_lines)
        cv2.line(line_image, (left_fitted[0], left_fitted[1]), (left_fitted[2], left_fitted[3]), BLUE, 10)
    if len(right_lines) > 0:
        right_fitted = fit_line(image, right_lines)
        cv2.line(line_image, (right_fitted[0], right_fitted[1]), (right_fitted[2], right_fitted[3]), BLUE, 10)            

    #create final image
    final = weighted_img(line_image, image)   
    
    #draw additional elements on image for development
    if DEBUG:
        #draw region
        cv2.line(image, tl, tr, WHITE, 2)
        cv2.line(image, tr, br, WHITE, 2)
        cv2.line(image, bl, br, WHITE, 2)
        cv2.line(image, bl, tl, WHITE, 2)
        #draw classified lines
        for line in left_lines:
            cv2.line(image, (int(line[0]),int(line[1])), (int(line[2]),int(line[3])), GREEN, 4)
        for line in right_lines:
            cv2.line(image, (int(line[0]),int(line[1])), (int(line[2]),int(line[3])), GREEN, 4)
        for line in outliers:
            cv2.line(image, (int(line[0]),int(line[1])), (int(line[2]),int(line[3])), RED, 4)    
        combined = combine(blur_gray,edges,image,final)
        return combined
        
    return final

        
files = ['solidWhiteCurve.jpg','solidWhiteRight.jpg','solidYellowCurve.jpg','solidYellowCurve2.jpg','solidYellowLeft.jpg','whiteCarLaneSwitch.jpg']
for file in files:       
    image = mpimg.imread('test_images/' + file)
    result = process(image)
    show_image(result, 'result')
    plt.savefig("test_images/processed_" + file)


## Test on Videos

You know what's cooler than drawing lanes over images? Drawing lanes over video!

We can test our solution on two provided videos:

`solidWhiteRight.mp4`

`solidYellowLeft.mp4`

In [ ]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [ ]:
def process_image(image):
    # NOTE: The output you return should be a color image (3 channel) for processing video below
    # TODO: put your pipeline here,
    # you should return the final output (image with lines are drawn on lanes)
    result = process(image)
    
    return result

Let's try the one with the solid white lane on the right first ...

In [ ]:
white_output = 'white.mp4'
clip1 = VideoFileClip("solidWhiteRight.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

Play the video inline, or if you prefer find the video in your filesystem (should be in the same directory) and play it in your video player of choice.

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

**At this point, if you were successful you probably have the Hough line segments drawn onto the road, but what about identifying the full extent of the lane and marking it clearly as in the example video (P1_example.mp4)?  Think about defining a line to run the full length of the visible lane based on the line segments you identified with the Hough Transform.  Modify your draw_lines function accordingly and try re-running your pipeline.**

Now for the one with the solid yellow lane on the left. This one's more tricky!

In [ ]:
yellow_output = 'yellow.mp4'
clip2 = VideoFileClip('solidYellowLeft.mp4')
yellow_clip = clip2.fl_image(process_image)
%time yellow_clip.write_videofile(yellow_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))

## Reflections

Congratulations on finding the lane lines!  As the final step in this project, we would like you to share your thoughts on your lane finding pipeline... specifically, how could you imagine making your algorithm better / more robust?  Where will your current algorithm be likely to fail?

Please add your thoughts below,  and if you're up for making your pipeline more robust, be sure to scroll down and check out the optional challenge video below!


Potential Improvements:
1. The fit line procedure averages out all the lines that are detected. It currently does not take the length of the line into account, but longer lines should weight more than shorter lines. Lines closer to the vehicle in generally are longer and should weight heavier than the lines further away.
2. The result videos are a little jittery, especially the challenge videos. We can take into account the slope of the left and right lane line detected in previous frame and average it out with the current detected slope.
3. Further tune the hsv, canny, and hough parameters. I belive there are still room to improve the result by tuning the parameters, however, we risk of overfitting if we tune these parameters excessively based on just three videos.

Potential Problems:
- HSV is applied in place of grayscale to handle the tree shadows in the challege videos. It has better tracking for the white and yellow objects, which produced better result in all three videos. However, if there are other white or yellow vehicles near by, they will affect the result.
- In all three videoes, the car is not following any cars too closer. If it is, the land lines in front will be blocked.

Image Illustration of potential problem. The top left image shows the result after HSV is applied. There are two vehicles in front, the dark vehicle is filtered out but the white vehicle is detected. On the top right image, Canny is applied, but the white vehicle remains. On the bottom left image, the trapezoid indicates the ROI (region of interest). If the white car is within the ROI, it will greatly distort the result.
<figure>
 <img src="hsv.jpg" width="600" alt="HSV result" />
 <figcaption>
 <p style="text-align: center;"> Image Illustration of potential problem.</p> 
 </figcaption>
</figure>



## Submission

If you're satisfied with your video outputs it's time to submit!  Submit this ipython notebook for review.


## Optional Challenge

Try your lane finding pipeline on the video below.  Does it still work?  Can you figure out a way to make it more robust?  If you're up for the challenge, modify your pipeline so it works with this video and submit it along with the rest of your project!

In [ ]:
challenge_output = 'extra.mp4'
clip2 = VideoFileClip('challenge.mp4')
challenge_clip = clip2.fl_image(process_image)
%time challenge_clip.write_videofile(challenge_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(challenge_output))